In [ ]:
!pip3 install gwcloud-python bilby

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import gwcloud_python

from pathlib import Path
import glob
import numpy as np
import bilby

# Recreating GW190521 eccentricity figure

In this tutorial, we will use the eccentricity posterior samples of a job stored in GWCloud to recreate Fig. 1 from __[GW190521 eccentricity](https://arxiv.org/pdf/2009.04771.pdf)__.

The first step is to authenticate with the GWCloud service by using your API token.

We also set a directory to be used to save and load files that are downloaded from GWCloud.

In [ ]:
from gwcloud_python import GWCloud, TimeRange

save_dir = Path('./EccentricHistogram')

### user token -- sign into gwcloud.org.au/auth/api-token.  Token is unique to individuals and provides authentication e.g., to LVC data and cluster resources
gwc = GWCloud(token='token goes here') 

The job analysing the eccentricity of GW190521 has a unique job name which we will use to filter our search of the GWCloud jobs.

Once we obtain the job we download the result files containing the posterior samples of the eccentricity analysis.

In [ ]:
# Download the merge_result.json files for the eccentric GW190521 run.
jobs = gwc.get_public_job_list(search = 'GW190521_eccentric', time_range = TimeRange.ANY)

for job in jobs:
    job.save_result_json_files(save_dir)

Now that we have obtained the JSON file containing the eccentricity posterior, we extract this information and plot it.

In [ ]:
for file in save_dir.glob('result/*.json'):
    eccentric_result = bilby.core.result.read_in_result(filename=str(file))
    
eccentric_result.posterior['log10_eccentricities'] = np.log10(eccentric_result.posterior['eccentricities'])

plt.hist(np.log10(eccentric_result.posterior['eccentricities']), bins=25, histtype='step', linewidth=3)
plt.xlim(-3, -0.6)
ax = plt.gca()
ax.axes.set_yticks([])
plt.xlabel(r'$\log_{10}e_{10}$')
plt.ylabel(r'$p(\log_{10}e_{10}|d)$')
plt.tight_layout()

plt.savefig(fname='GW190521_eccentricity_posterior_histogram.pdf')